In [3]:
# =============================
# STEP 1: Mount Google Drive
# =============================
from google.colab import drive
drive.mount('/content/drive')

# Path to your dataset folder in Drive
data_path = "/content/drive/MyDrive/APR"
train_path = f"{data_path}/mnist_train.csv"
test_path = f"{data_path}/mnist_test.csv"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# =============================
# STEP 2: Import Libraries
# =============================
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
from PIL import Image

# =============================
# STEP 3: Load the Dataset
# =============================
print("Loading data...")
train = pd.read_csv(train_path)
test = pd.read_csv(test_path)
print("Data loaded successfully!")

Loading data...
Data loaded successfully!


In [5]:
# =============================
# STEP 4: Prepare Data
# =============================
# Separate labels and pixel values
x_train = train.iloc[:, 1:].values
y_train = train.iloc[:, 0].values

x_test = test.iloc[:, 1:].values
y_test = test.iloc[:, 0].values

# Normalize pixel values
x_train = x_train / 255.0
x_test = x_test / 255.0

# Reshape to (28,28,1)
x_train = x_train.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1)

print(f"Training samples: {x_train.shape}, Test samples: {x_test.shape}")

Training samples: (60000, 28, 28, 1), Test samples: (10000, 28, 28, 1)


In [6]:
# =============================
# STEP 5: Build CNN Model
# =============================
model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(28,28,1)),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [7]:
# =============================
# STEP 6: Train the Model
# =============================
history = model.fit(x_train, y_train, epochs=5, validation_split=0.1)

Epoch 1/5
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 12s 5ms/step - accuracy: 0.8969 - loss: 0.3384 - val_accuracy: 0.9878 - val_loss: 0.0468
Epoch 2/5
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9854 - loss: 0.0486 - val_accuracy: 0.9855 - val_loss: 0.0483
Epoch 3/5
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9906 - loss: 0.0289 - val_accuracy: 0.9873 - val_loss: 0.0415
Epoch 4/5
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9930 - loss: 0.0231 - val_accuracy: 0.9908 - val_loss: 0.0364
Epoch 5/5
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.9949 - loss: 0.0164 - val_accuracy: 0.9893 - val_loss: 0.0401


In [8]:
# =============================
# STEP 7: Evaluate the Model
# =============================
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f"✅ Test accuracy: {test_acc:.4f}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9808 - loss: 0.0561
✅ Test accuracy: 0.9866


In [9]:
# =============================
# STEP 8: Save the Model
# =============================
model.save("/content/drive/MyDrive/APR/digit_recognition_model.h5")
print("Model saved to Google Drive!")

Model saved to Google Drive!


In [11]:
# =============================
# STEP 9: Predict Custom Image
# =============================

def predict_digit(image_path):
    """Predicts a single digit (0–9) from a given image file."""
    img = Image.open(image_path).convert('L')  # convert to grayscale
    img = img.resize((28, 28))                 # resize to 28x28
    img = np.array(img)
    img = 255 - img                            # invert if needed (white background)
    img = img / 255.0
    img = img.reshape(1, 28, 28, 1)

    prediction = model.predict(img)
    digit = np.argmax(prediction)
    confidence = np.max(prediction)
    print(f"Predicted digit: {digit} (confidence: {confidence:.2f})")

    plt.imshow(img.reshape(28, 28), cmap='gray')
    plt.title(f"Prediction: {digit}")
    plt.axis('off')
    plt.show()

# Example: Predict from an image in your Drive
# predict_digit("/content/drive/MyDrive/APR/sample_digit.png")